In [1]:
import pandas as pd

from reportlab.graphics.barcode import code39, code128, code93, eanbc, qr, usps
from reportlab.graphics.shapes import Drawing 
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm, cm
from reportlab.pdfgen.canvas import Canvas
from reportlab.graphics import renderPDF

In [2]:
date = '2021-05-05'

In [3]:
df = pd.read_csv(f'products-3dxtech-{date}.csv')
df

,code,pairCode,name,ean,productNumber,stock,stickers remaining,current stock,sticker name
0,970,NaN,"3DXSTAT™ ESD PLA 3D printing filament 1,75mm 1kg",8594199700121,NaN,0,20,0.0,3DXTECH\n3DXSTAT\nESD PLA\n1.75 mm 1 kg
1,971,NaN,"3DXSTAT™ ESD PLA 3D printing filament 2,85 mm 1kg",8594199700138,NaN,0,10,0.0,3DXTECH\n3DXSTAT\nESD PLA\n2.85 mm 1 kg
2,0,NaN,CARBONX™ Carbon Fiber Nylon 3D Printing Filame...,8594199700541,NaN,0,10,NaN,3DXTECH\nCARBONX\nCarbon Fiber Nylon\n1.75 mm ...
3,2783,NaN,FLUORX™ 3d printing filament made using KYNAR®...,8594199700220,NaN,0,10,NaN,3DXTECH\nFLUORX\nPVDF KYNAR\n1.75 mm 0.5 kg


In [4]:
def generate_values_from_df(df):
    eans = []
    texts = []
    for i, row in df.iterrows():
        count = row['stickers remaining']
        eans.extend(count * [row['ean']])
        texts.extend(count * [row['sticker name']])
    
    return eans, texts

eans, texts = generate_values_from_df(df)

In [5]:
# eans = 6*["8594199700220"]
# text = """
# 3DXTECH
# AMIDEX PA6-GF30
# Glass Fiber Nylon
# 1,75 mm 0,75 kg
# """
show_border = False
file_path = f'3dxtech_{date}.pdf'

page_size = A4
canvas = Canvas(file_path, pagesize=page_size)

# both top and bottom
y_margin = 4.5 * mm

x_count, y_count = 3, 8
items_per_page = x_count * y_count
# size of one label
# 70, 36
x_size = page_size[0] / x_count
y_size = (page_size[1] - 2 * y_margin) / y_count

# size of one bar code
code_width = 37 * mm
code_height = 26 * mm
i = 0

for i, (ean, text) in enumerate(zip(eans, texts)):
    index_on_page = (i % items_per_page)
    row = index_on_page // x_count
    col = index_on_page % x_count

    if index_on_page == 0 and i > 0:
        canvas.showPage()

    if index_on_page == 0:
        canvas.translate(0, y_margin)
    
    if not ean:
        continue
        
    canvas.saveState()
    
    # fill columns from top
    canvas.translate(x_size*col, y_size*(y_count-1-row))

    if show_border:
        canvas.rect(0, 0, x_size, y_size)

    barcode_eanbc13 = eanbc.Ean13BarcodeWidget(ean)
    bounds = barcode_eanbc13.getBounds()
    drawing = Drawing(code_width, code_height)
    drawing.add(barcode_eanbc13)
    renderPDF.draw(drawing, canvas,
                   2 * mm,
                   (y_size - code_height) / 2)

    img_height = 0.7*cm
    img_width = 400/120*img_height
    img_bottom = y_size - img_height - (y_size - code_height) / 2
    canvas.drawImage('s3dt-logo-en-black-on-white-800-240.png',
                     code_width + 2 * mm,
                     img_bottom,
                     width=img_width,
                     height=img_height)

    textobject = canvas.beginText()
    textobject.setTextOrigin(code_width + 2 * mm, img_bottom - 5 * mm)
    textobject.setFont("Helvetica", 9)
    textobject.textLines(text.strip())
    canvas.drawText(textobject)

    canvas.restoreState()
    
canvas.save()

In [6]:
!open {file_path}